In [ ]:
from gensim.models import word2vec
import pandas as pymysql
from gensim.models import KeyedVectors
import gensim
import numpy as np

def create_feature_vectors(corpus, doc_id_list):
#     here we use the pre trained model for creating feature vectors
    model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    # word_vector = []
    # word_not_in_vocab = []
    for doc in corpus:
        # print(doc)
        word_vector = []
        # word_vector = [model[x] for x in doc.split(' ')]
        for x in doc.split(' '):
            try:
                word_vector.append(model[x])
                # print("---"+str(len(model[x])))
            except KeyError:
                # print("word "+str(x)+" not in vocab")
                # word_not_in_vocab.append(x)
                word_vector.append(np.array([0] * 300))
    # create a list of zeroes

    # print(word_vector)
    #     this word vector is for each document, we need to figure out how to use this word vector further
    # print(word_not_in_vocab)

host = 'webirfacts.csdsyolim34i.us-east-2.rds.amazonaws.com'
port = 3306
username = 'group5'
password = 'aws12345'
dbname = 'facts_8'

# setting up a connection to aws
conn = pymysql.connect(host, user=username,port=port, passwd=password, db=dbname)
conn_cursor = conn.cursor()



# we need to create a corpus first
#get the cleaned extracted text from image from the database
get_clean_data = 'select doc_id, clean_data from clean_text where clean_data is not null or length(clean_data) > 0'
clean_text_df = pandas.read_sql(get_clean_data, con = conn)

corpus = []
doc_id_list = []
for data in clean_text_df.itertuples():
    corpus.append(data[2])
    doc_id_list.append(data[1])
# pass this corpus to the feature vector generator
# create_feature_vectors(corpus)
create_feature_vectors(['this is number 17'], doc_id_list)
# print(len(corpus))

In [24]:
print(len(corpus))

2664


In [25]:
type(corpus)

list

In [31]:
corpus[110]

'man committed suicide golden gate bridge left note i going walk bridge one person smile way'

In [192]:
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

# Stemming and tokenizing initial data

Stemming is changing the word to the normal form like "finished" -> "finish"; "cats" -> "cat"

In [35]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [37]:
stopwords = nltk.corpus.stopwords.words('english')

In [40]:
print (stopwords[:4])

['i', 'me', 'my', 'myself']


In [41]:
from nltk.stem.snowball import SnowballStemmer

In [42]:
stemmer = SnowballStemmer("english")

Below I define two functions:

    tokenize_and_stem: tokenizes (splits the facts into a list of its respective tokens (words) and also stems each token

    tokenize_only: tokenizes the data only
I use both these functions to create a dictionary but later convert stems back to their full words for presentation purposes.

In [49]:
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [51]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in corpus:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [57]:
print(totalvocab_tokenized[100:110])
print(totalvocab_stemmed[100:110])

['contain', 'melatonin', 'help', 'regulate', 'body', 'natural', 'rhythm', 'men', 'masturbate', 'day']
['contain', 'melatonin', 'help', 'regul', 'bodi', 'natur', 'rhythm', 'men', 'masturb', 'day']


In [59]:
vocab_frame = pandas.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 27763 items in vocab_frame


In [61]:
vocab_frame.head()

,words
averag,average
woman,woman
spend,spends
year,year
life,life


# Tf-idf and document similarity

Here, I define term frequency-inverse document frequency (tf-idf) vectorizer parameters and then convert the facts list into a tf-idf matrix.

To get a Tf-idf matrix, first count word occurrences by document. This is transformed into a document-term matrix (dtm).

Then apply the term frequency-inverse document frequency weighting: words that occur frequently within a document but not frequently within the corpus receive a higher weighting as these words are assumed to contain more meaning in relation to the document.

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [86]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=27000,
                                 min_df=0.03, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

max_df: this is the maximum frequency within the documents a given feature can have to be used in the tfi-idf matrix. If the term is in greater than 80% of the documents it probably cares little meanining in our data

min_idf: Here I pass 0.2; the term must be in at least 20% of the document. You can play with this parameter to change the number of clusters

ngram_range: this just means I'll look at unigrams, bigrams and trigrams.

In [87]:
%time tfidf_matrix = tfidf_vectorizer.fit_transform(corpus) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

Wall time: 1.48 s
(2664, 23)


C:\Users\Васек\AppData\Roaming\Python\Python35\site-packages\sklearn\feature_extraction\text.py:1039: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [89]:
#terms is just a list of the features used in the tf-idf matrix. This is a vocabulary
terms = tfidf_vectorizer.get_feature_names()

dist is defined as 1 - the cosine similarity of each document. Cosine similarity is measured against the tf-idf matrix and can be used to generate a measure of similarity between each document and the other documents in the corpus (each doc among all docs). Subtracting it from 1 provides cosine distance which I will use for plotting.

In [90]:
from sklearn.metrics.pairwise import cosine_similarity

In [91]:
dist = 1 - cosine_similarity(tfidf_matrix)

# K-means Clustering

In [92]:
from sklearn.cluster import KMeans

In [153]:
#can change the number of clusters you want
num_clusters = 12

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

Wall time: 463 ms


I use joblib.dump to pickle the model, once it has converged and to reload the model/reassign the labels as the clusters.

In [154]:
from sklearn.externals import joblib

In [155]:
#uncomment the below to save your model
#joblib.dump(km,  'doc_cluster.pkl')

['doc_cluster.pkl']

In [156]:
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [ ]:
deb_file = pandas.read_csv('clean_data.csv', sep=';')
doc_id = np.array(deb_file[deb_file.columns[0]])
clean_data = np.array(deb_file[deb_file.columns[1]])

In [249]:
doc_id_str = np.array(deb_file[deb_file.columns[0]].apply(str))
doc_id_str[:5]

array(['1239', '2586', '50', '2544', '591'], dtype=object)

Here, I create a dictionary of doc_id, the fact and the cluster assignment.

In [158]:
text = {'doc_id': doc_id, 'fact': clean_data, 'cluster': clusters}

frame = pandas.DataFrame(text, index = [clusters] , columns = ['doc_id', 'fact', 'cluster'])

In [159]:
frame.head()

,doc_id,fact,cluster
1,1239,average woman spends 17 year life dieting,1
0,2586,in chinese city chongqing cell phone addict si...,0
6,50,people say 1 million actually around like 8 fact,6
3,2544,lighteyed people experience le pain childbirth...,3
11,591,iceland us geothermal heat pull almost 100 per...,11


In [160]:
frame['cluster'].value_counts() #number of facts per cluster (clusters from 0 to 4)

0     1344
6      164
3      154
8      140
11     139
10     120
2      116
5      115
9      102
1      102
4       94
7       74
Name: cluster, dtype: int64

Here is indexing and sorting on each cluster to identify which are the top n (I chose n=10) words that are nearest to the cluster centroid. This provides us the main topic of the cluster.

In [161]:
from __future__ import print_function

In [171]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :10]: #replace 10 with any number of words words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d doc_id:" % i, end='')
    for doc_id in frame.ix[i]['doc_id'].values.tolist():
        print(' %s,' % doc_id, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: b'person', b'dollar', b'time', b'fact', b'eat', b'study', b'food', b'researchers', b'le', b'feel',

Cluster 0 doc_id: 2586, 2461, 1371, 2530, 1117, 892, 1605, 2517, 286, 1125, 32, 1181, 449, 247, 456, 1151, 1724, 1587, 2235, 200, 1326, 455, 2172, 11, 1581, 713, 2632, 1751, 1536, 661, 1841, 622, 2635, 1219, 2156, 1718, 427, 2574, 1308, 1442, 385, 2288, 1572, 2566, 60, 2169, 1710, 2624, 2105, 2324, 583, 2466, 1599, 803, 651, 2657, 735, 360, 1070, 2071, 2615, 476, 1736, 2125, 693, 1991, 742, 855, 2307, 615, 488, 1368, 1539, 1691, 497, 1135, 151, 2509, 637, 1170, 1793, 501, 1912, 522, 1359, 2311, 911, 1299, 905, 1341, 2017, 1887, 1217, 1659, 9, 494, 1434, 1361, 164, 400, 344, 304, 1684, 2014, 1168, 1494, 442, 874, 1666, 1781, 2319, 1242, 2448, 632, 2129, 2499, 2042, 78, 1234, 1650, 2357, 463, 2368, 2502, 1985, 53, 8, 2219, 328, 1957, 1023, 2630, 579, 1837, 2351, 740, 2247, 1124, 236, 1109, 1054, 74, 1036, 1497, 1314, 1061, 1380, 1843, 2447, 1126, 66

C:\Users\Васек\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Васек\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


 b'help', b'year', b'fact', b'people',

Cluster 9 doc_id: 2063, 1671, 88, 1662, 1417, 703, 844, 2587, 178, 794, 739, 271, 373, 698, 680, 1765, 2318, 381, 1621, 704, 722, 2494, 2040, 52, 2012, 1695, 826, 221, 1399, 635, 1872, 399, 1990, 203, 1790, 25, 994, 750, 590, 2476, 2383, 172, 732, 1311, 239, 2081, 1689, 70, 19, 626, 617, 2099, 2008, 1329, 1466, 1713, 1972, 580, 521, 762, 1826, 1707, 976, 999, 721, 1747, 608, 812, 775, 1890, 716, 904, 780, 2300, 562, 1347, 2569, 1956, 1808, 539, 1603, 598, 1435, 34, 1729, 1974, 886, 2022, 503, 981, 417, 2401, 2650, 1460, 2626, 391, 1994, 757, 1680, 1017, 1938, 2618,

Cluster 10 words: b'make', b'feel', b'people', b'study', b'food', b'like', b'eat', b'according', b'le', b'researchers',

Cluster 10 doc_id: 2141, 461, 1711, 542, 1876, 930, 1812, 1803, 2589, 759, 568, 361, 2389, 209, 1788, 912, 1702, 878, 1640, 1134, 1693, 870, 777, 1797, 741, 1575, 2369, 1096, 2395, 1989, 1593, 1676, 730, 2422, 644, 2363, 1116, 2123, 2159, 162, 287, 191, 496, 1723, 1

# Multidimensional scaling

I do not use PCA on purpose, but we can try it also 

In [173]:
import os  # for os.path.basename

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]

# Some vizualization (for the final report)

I based the cluster names off the words that were closest to each cluster centroid.

In [190]:
#set up colors per clusters using a dict
cluster_colors = {0: '#FF4119', 1: '#FFEE19', 2: '#B9FF19', 3: '#19FFDC', 4: '#1938FF',
                 5 : '#D219FF', 6: '#FF19CB', 7: '#A4001E', 8: '#009CA4', 9: '#00A443',
                 10: '#9FA400', 11:'#A47700'}

#set up cluster names using a dict
cluster_names = {0: 'person, dollar, time', 
                 1: 'year, woman, people', 
                 2: 'day, person, dollar', 
                 3: 'study, researches, dollar', 
                 4: 'brain, people, time',
                 5: 'used, study, woman',
                 6: 'study, according, researches',
                 7: 'time, year, make',
                 8: 'man, woman, like',
                 9: 'human, brain, help',
                 10: 'feel, food, eat',
                 11: 'percent, people, study',}

### Some javascript for Delyana

In [193]:
#define custom toolbar location
class TopToolbar(mpld3.plugins.PluginBase):
    """Plugin for moving toolbar to top of figure"""

    JAVASCRIPT = """
    mpld3.register_plugin("toptoolbar", TopToolbar);
    TopToolbar.prototype = Object.create(mpld3.Plugin.prototype);
    TopToolbar.prototype.constructor = TopToolbar;
    function TopToolbar(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };

    TopToolbar.prototype.draw = function(){
      // the toolbar svg doesn't exist
      // yet, so first draw it
      this.fig.toolbar.draw();

      // then change the y position to be
      // at the top of the figure
      this.fig.toolbar.toolbar.attr("x", 150);
      this.fig.toolbar.toolbar.attr("y", 400);

      // then remove the draw function,
      // so that it is not called again
      this.fig.toolbar.draw = function() {}
    }
    """
    def __init__(self):
        self.dict_ = {"type": "toptoolbar"}

In [232]:
#create data frame that has the result of the MDS plus the cluster numbers and doc_ids
df = pandas.DataFrame(dict(x=xs, y=ys, label=clusters, doc_id=doc_id))
#group by cluster
groups = df.groupby('label')

#define custom css to format the font and to remove the axis labeling
css = """
text.mpld3-text, div.mpld3-tooltip {
  font-family:Arial, Helvetica, sans-serif;
}

g.mpld3-xaxis, g.mpld3-yaxis {
display: none; }

svg.mpld3-figure {
margin-left: -100px;}
"""

# Plot 
fig, ax = plt.subplots(figsize=(12,14)) #set plot size
ax.margins(0.01) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=10, 
                     label=cluster_names[name], mec='none', 
                     color=cluster_colors[name])
    ax.set_aspect('auto')
    labels = [i for i in group.doc_id]
    
    #set tooltip using points, labels and the already defined 'css'
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], labels,
                                       voffset=10, hoffset=10, css=css)
    #connect tooltip to fig
    mpld3.plugins.connect(fig, tooltip, TopToolbar())    
    
    #set tick marks as blank
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #set axis as blank
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    
ax.legend(numpoints=1) #show legend with only one dot

mpld3.display() #show the plot

#uncomment the below to export to html
#html = mpld3.fig_to_html(fig)
#print(html)

# Hierarchical document clustering

In [233]:
from scipy.cluster.hierarchy import ward, dendrogram

# Ward

In [285]:
linkage_matrix = ward(dist) #define the linkage_matrix using ward clustering pre-computed distances

fig, ax = plt.subplots(figsize=(15, 20)) # set size
ax = dendrogram(linkage_matrix, orientation="right", labels=doc_id_str);

plt.tick_params(\
    axis= 'x',          
    which='both',      
    bottom='off',      
    top='off',         
    labelbottom='off')

plt.tight_layout() 

plt.savefig('ward_clusters.png', dpi=200)

In [255]:
plt.close()

### LDA

LDA is a probabilistic topic model that assumes documents are a mixture of topics and that each word in the document is attributable to the document's topics.

In [299]:
from gensim import corpora, models, similarities

In [296]:
#tokenize
%time tokenized_text = [tokenize_and_stem(text) for text in corpus]

#remove stop words
%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

Wall time: 1.28 s
Wall time: 93.7 ms


In [313]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
new_corpus = [dictionary.doc2bow(text) for text in texts]

In [328]:
%time lda = models.LdaModel(new_corpus, num_topics=12)

Wall time: 1.69 s


In [329]:
lda.show_topics()

[(10,
  '0.021*"19" + 0.018*"17" + 0.012*"96" + 0.011*"161" + 0.010*"159" + 0.010*"34" + 0.009*"262" + 0.008*"5" + 0.008*"1107" + 0.008*"503"'),
 (9,
  '0.018*"56" + 0.014*"2157" + 0.014*"2156" + 0.011*"4" + 0.010*"99" + 0.010*"262" + 0.010*"19" + 0.008*"161" + 0.008*"51" + 0.006*"103"'),
 (3,
  '0.011*"99" + 0.010*"4" + 0.010*"19" + 0.010*"125" + 0.009*"1349" + 0.008*"898" + 0.008*"397" + 0.007*"116" + 0.007*"174" + 0.007*"332"'),
 (7,
  '0.012*"73" + 0.010*"96" + 0.010*"51" + 0.010*"39" + 0.009*"1097" + 0.009*"458" + 0.008*"701" + 0.008*"126" + 0.007*"16" + 0.006*"19"'),
 (2,
  '0.018*"369" + 0.018*"19" + 0.014*"17" + 0.009*"847" + 0.007*"414" + 0.007*"95" + 0.007*"64" + 0.007*"247" + 0.006*"125" + 0.006*"34"'),
 (4,
  '0.017*"19" + 0.016*"110" + 0.015*"126" + 0.013*"17" + 0.009*"56" + 0.009*"96" + 0.008*"252" + 0.008*"700" + 0.008*"255" + 0.007*"397"'),
 (1,
  '0.013*"19" + 0.012*"184" + 0.011*"17" + 0.011*"99" + 0.009*"34" + 0.009*"5" + 0.009*"26" + 0.009*"51" + 0.008*"194" + 0.007

Here the numbers are the indexis of terms, not doc_id